In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from fastprogress import progress_bar
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd
from PIL import Image
import re
import shutil
import urllib.request

In [3]:
images_path = '../data/user_images/'
labels_path = '../data/user_labels.csv'
target_size = 640

In [4]:
!rm -rf ../data/user_images/*

In [5]:
df = pd.DataFrame()
i, step = 0, 500
while True:
    print("New batch...")
    batch = pd.read_json(
        ""
    ).dropna()
    i += step
    
    if batch.empty:
        break
    
    if df.empty:
        df = batch
    else: 
        df = df.append(batch)

New batch...
New batch...
New batch...
New batch...
New batch...
New batch...
New batch...


In [6]:
def get_floats_from_stringy_tup(string):
    p = re.compile(r'\d+\.\d+|\d+')   
    return p.findall(string)

def split_array(array):
    return float(array[0]), target_size-float(array[1])

for col in list(df.columns[1:12]):
    df[col] = df[col].apply(get_floats_from_stringy_tup)
    df[col+'_x'], df[col+'_y'] = zip(*df[col].map(split_array))
    df = df.drop(col, axis=1)

uploadImageDf = pd.DataFrame(df['uploadedImage'].values.tolist())
df['hash'] = uploadImageDf['hash'].values

In [7]:
for i in progress_bar(range(df.shape[0])):
    urllib.request.urlretrieve(
        df.iloc[i]['uploadedImage']['url'], 
        f"{images_path}/{df.iloc[i]['hash']}.png"
    )

In [8]:
df = df.drop('uploadedImage', axis=1)

def add_png(hashes):
    return hashes + '.png'

df['hash'] = df['hash'].apply(add_png)

df = df.rename(columns={"hash":"filename"})
df = df[df.columns[-23:]]

In [19]:
df

,leftEyePoint_x,leftEyePoint_y,rightEyePoint_x,rightEyePoint_y,centreMouthPoint_x,centreMouthPoint_y,leftMouthPoint_x,leftMouthPoint_y,rightMouthPoint_x,rightMouthPoint_y,...,chinPoint_y,leftHeadPoint_x,leftHeadPoint_y,rightHeadPoint_x,rightHeadPoint_y,leftEarPoint_x,leftEarPoint_y,rightEarPoint_x,rightEarPoint_y,filename
0,176.231884,319.227053,417.906594,325.668261,352.979058,488.502415,294.235097,517.874396,401.417054,502.930749,...,620.933962,124.186788,513.236715,471.497585,484.380024,106.151353,275.684372,432.850242,249.404179,6310b3dbaa3549ea90badf31c7bee933.png
1,169.017706,206.119155,372.045073,254.557150,224.669879,480.772947,185.507246,487.471812,262.286619,499.323671,...,533.848623,96.360701,304.541063,409.661836,355.555556,154.589372,119.549106,438.518503,192.721401,d65b1920c7ea4c61bd7ab8709b7e6d1c.png
2,212.560386,194.009662,317.681159,231.111111,222.608696,300.676329,193.752005,310.209324,247.342995,324.122368,...,341.127198,172.367150,211.014493,355.555556,285.990338,224.669879,48.437995,432.334928,181.384855,943a14d469664950a9d4168eee6e2291.png
3,267.954904,287.536232,408.631232,272.592585,364.315604,481.803527,328.760049,484.380024,395.748792,479.227053,...,539.001595,194.267295,340.096618,468.921087,343.703696,164.380024,39.162633,431.819638,14.428334,e7fbe08bb2fe48149ffd829a4a4ddd00.png
4,198.904976,201.996764,290.628019,197.359083,250.434783,312.785821,219.516908,320.515290,275.169082,313.816425,...,357.616739,148.405797,288.566812,349.887271,265.893720,110.789034,135.523334,385.442826,90.177126,b9db262dc66f4ba0b529559f64e0d7d6.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,239.583333,184.715447,337.920000,193.706667,281.666667,274.037940,259.166667,287.913279,303.333333,287.479675,...,310.027100,145.000000,201.626016,396.250000,221.571816,145.066667,28.160000,443.333333,36.856369,25fe971f71fc44a681ed9408e4aa087b.png
101,206.250000,219.403794,419.840000,148.480000,386.275547,345.884427,327.680000,391.395547,457.955547,351.004427,...,385.040650,6.250000,427.967480,582.916667,213.766938,7.083333,115.772358,494.583333,3.035230,0378c4152f5f48a08a8f6401ba223553.png
116,214.186667,282.453333,314.880000,314.880000,237.226667,367.786667,217.600000,368.640000,252.083333,375.501355,...,434.471545,92.500000,255.826558,393.750000,393.279133,178.750000,150.460705,446.666667,254.959350,4cfefafb75c54a2d80097fc296d46753.png
152,195.416667,228.075881,379.166667,236.747967,341.250000,404.552846,288.750000,419.728997,395.833333,399.783198,...,470.460705,40.833333,333.441734,486.666667,264.065041,73.333333,110.569106,427.500000,74.579946,0f47f0db5a4b449ab608f524603c3b6a.png


In [20]:
df.to_csv(labels_path, index=False)

In [53]:
!rm -rf '../data/resized_and_user_images/*'

for src in ['user', 'resized']:
    src_files = os.listdir(f'../data/{src}_images/')
    for file_name in progress_bar(src_files):
        full_file_name = os.path.join(f'../data/{src}_images/', file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, '../data/resized_and_user_images')

In [43]:
user_df = pd.read_csv(f'../data/user_labels.csv')
resized_df = pd.read_csv(f'../data/resized_labels.csv')

In [44]:
resized_df

,filename,0_x,0_y,1_x,1_y,2_x,2_y,3_x,3_y,4_x,...,6_x,6_y,7_x,7_y,8_x,8_y,9_x,9_y,10_x,10_y
0,Abyssinian_1.jpg,180.800003,104.799995,180.800003,62.399998,202.666656,80.800003,223.466660,61.599998,222.399994,...,193.066681,97.600006,209.066650,98.400002,195.200012,119.199997,198.933334,115.200005,205.333328,119.199997
1,Abyssinian_100.jpg,126.700508,126.720001,153.502533,65.279999,198.984772,80.000000,268.020294,50.560001,263.147217,...,185.177673,122.880005,237.969543,118.400002,205.482239,162.559998,222.538071,156.800003,237.157364,160.000000
2,Abyssinian_102.jpg,60.799999,161.720428,17.920000,55.053764,125.439995,74.322578,202.880005,22.709677,203.519989,...,101.120003,144.516129,160.639984,138.322586,115.200005,211.957001,135.679993,202.322571,155.520004,205.763428
3,Abyssinian_103.jpg,171.520004,150.427353,155.520004,92.079773,193.279999,93.903130,226.559998,64.729340,231.040009,...,190.079987,142.222229,215.680008,134.928772,202.880005,175.954422,208.639999,173.219376,216.320007,171.396011
4,Abyssinian_105.jpg,160.639984,145.919998,152.959991,90.453331,195.200012,90.453331,237.440002,90.453331,228.479996,...,180.480011,155.306671,209.920013,155.306671,187.520004,191.146667,194.559998,190.293335,201.600006,192.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5984,yorkshire_terrier_93.jpg,82.836876,137.600006,105.531914,110.080002,178.156036,81.279999,256.453888,104.960007,276.879425,...,147.517731,113.919998,216.737595,117.119995,144.113480,144.639999,174.751770,138.879990,201.985809,144.639999
5985,yorkshire_terrier_96.jpg,108.160004,81.194031,135.679993,37.253731,193.919998,20.059702,227.199997,69.731346,245.119995,...,159.360001,77.373131,200.319992,94.567169,153.599991,139.462692,175.359985,138.507462,189.440002,151.880600
5986,yorkshire_terrier_97.jpg,131.820892,138.879990,139.462692,77.440002,199.641800,101.759995,278.925385,88.959999,250.268646,...,175.761185,139.519989,219.701508,140.160004,178.626862,172.160004,196.776123,173.440002,207.283569,172.160004
5987,yorkshire_terrier_98.jpg,129.919998,204.799988,147.199997,149.333328,170.240005,181.760010,202.240005,159.573334,204.160004,...,156.800003,215.039993,182.399994,217.600006,155.520004,254.293335,168.959991,251.733337,176.639999,253.440002


In [45]:
resized_df[resized_df.columns[1:]] = resized_df[resized_df.columns[1:]].apply(lambda x: x/2)

cols = list(resized_df.columns)
cols = cols[1:] + [cols[0]]
resized_df = resized_df[cols]

resized_df.columns = user_df.columns

In [46]:
resized_and_user_df = user_df.append(resized_df)

In [47]:
resized_and_user_df

,leftEyePoint_x,leftEyePoint_y,rightEyePoint_x,rightEyePoint_y,centreMouthPoint_x,centreMouthPoint_y,leftMouthPoint_x,leftMouthPoint_y,rightMouthPoint_x,rightMouthPoint_y,...,chinPoint_y,leftHeadPoint_x,leftHeadPoint_y,rightHeadPoint_x,rightHeadPoint_y,leftEarPoint_x,leftEarPoint_y,rightEarPoint_x,rightEarPoint_y,filename
0,176.231884,319.227053,417.906594,325.668261,352.979058,488.502415,294.235097,517.874396,401.417054,502.930749,...,620.933962,124.186788,513.236715,471.497585,484.380024,106.151353,275.684372,432.850242,249.404179,6310b3dbaa3549ea90badf31c7bee933.png
1,169.017706,206.119155,372.045073,254.557150,224.669879,480.772947,185.507246,487.471812,262.286619,499.323671,...,533.848623,96.360701,304.541063,409.661836,355.555556,154.589372,119.549106,438.518503,192.721401,d65b1920c7ea4c61bd7ab8709b7e6d1c.png
2,212.560386,194.009662,317.681159,231.111111,222.608696,300.676329,193.752005,310.209324,247.342995,324.122368,...,341.127198,172.367150,211.014493,355.555556,285.990338,224.669879,48.437995,432.334928,181.384855,943a14d469664950a9d4168eee6e2291.png
3,267.954904,287.536232,408.631232,272.592585,364.315604,481.803527,328.760049,484.380024,395.748792,479.227053,...,539.001595,194.267295,340.096618,468.921087,343.703696,164.380024,39.162633,431.819638,14.428334,e7fbe08bb2fe48149ffd829a4a4ddd00.png
4,198.904976,201.996764,290.628019,197.359083,250.434783,312.785821,219.516908,320.515290,275.169082,313.816425,...,357.616739,148.405797,288.566812,349.887271,265.893720,110.789034,135.523334,385.442826,90.177126,b9db262dc66f4ba0b529559f64e0d7d6.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5984,41.418438,68.800003,52.765957,55.040001,89.078018,40.639999,128.226944,52.480003,138.439713,66.239998,...,56.959999,108.368797,58.559998,72.056740,72.320000,87.375885,69.439995,100.992905,72.320000,yorkshire_terrier_93.jpg
5985,54.080002,40.597015,67.839996,18.626865,96.959999,10.029851,113.599998,34.865673,122.559998,62.089554,...,38.686565,100.159996,47.283585,76.799995,69.731346,87.679993,69.253731,94.720001,75.940300,yorkshire_terrier_96.jpg
5986,65.910446,69.439995,69.731346,38.720001,99.820900,50.879997,139.462692,44.480000,125.134323,68.160004,...,69.759995,109.850754,70.080002,89.313431,86.080002,98.388062,86.720001,103.641785,86.080002,yorkshire_terrier_97.jpg
5987,64.959999,102.399994,73.599998,74.666664,85.120003,90.880005,101.120003,79.786667,102.080002,105.813332,...,107.519997,91.199997,108.800003,77.760002,127.146667,84.479996,125.866669,88.320000,126.720001,yorkshire_terrier_98.jpg


In [48]:
resized_and_user_df.to_csv('../data/resized_and_user_labels.csv', index=False)